In [14]:
import pandas as pd
import os

def concatenate_and_standardize_csv_files(folder_path, output_filename="entity_alignment.csv"):
    """
    Concatenates multiple CSV files from a specified folder, standardizes column names,
    sorts by 'index_id', and saves the result to a new CSV file.

    Args:
        folder_path (str): The path to the folder containing the CSV files.
        output_filename (str): The name of the output CSV file.
    """
    standard_columns = ['index_id', 'candidate_id', 'confidence_score', 'need_judge']
    all_dataframes = []

    # Get all CSV file paths in the specified folder
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.csv')]

    if not csv_files:
        print(f"Nu s-au găsit fișiere CSV în directorul: {folder_path}. Asigurați-vă că fișierele sunt prezente.")
        return

    for file_path in csv_files:
        try:
            # Read the file with no header initially to find the correct header row
            df_raw = pd.read_csv(file_path, header=None, encoding='utf-8')

            # Look for the header row containing all standard column names
            header_row_index = -1
            for i, row in df_raw.iterrows():
                # Convert row to string and check for standard column names
                row_values = row.dropna().astype(str).tolist()
                row_str = " ".join(row_values).lower()
                
                # Check if all standard column names (or close variations) are present
                if all(any(col_keyword in s for s in row_values) for col_keyword in [col.lower() for col in standard_columns]):
                    header_row_index = i
                    break
            
            if header_row_index == -1:
                print(f"Avertisment: Antetul standard nu a fost găsit explicit în {file_path}. Se încearcă inferența coloanelor.")
                # Fallback: Try to infer based on typical data start
                potential_data_start_row = -1
                for i, row in df_raw.iterrows():
                    first_val = row.dropna().iloc[0] if not row.dropna().empty else None
                    if pd.notna(first_val) and (isinstance(first_val, (int, float)) or str(first_val).replace('.', '').isdigit() or str(first_val).lower() in ['true', 'false']):
                        potential_data_start_row = i
                        break
                
                if potential_data_start_row != -1:
                    df = pd.read_csv(file_path, header=potential_data_start_row, encoding='utf-8')
                    # Assume column order based on typical CSV structure if header not found
                    if len(df.columns) >= len(standard_columns):
                        df.rename(columns={df.columns[0]: 'index_id',
                                           df.columns[1]: 'candidate_id',
                                           df.columns[2]: 'confidence_score',
                                           df.columns[3]: 'need_judge'}, inplace=True)
                        df = df[standard_columns]
                    else:
                        print(f"Se omite {file_path} din cauza numărului insuficient de coloane după inferență.")
                        continue
                else:
                    print(f"Se omite {file_path} deoarece nu a fost găsit un antet sau un început de date identificabil.")
                    continue
            else:
                # Read the file again with the identified header row
                df = pd.read_csv(file_path, header=header_row_index, encoding='utf-8')

                # Standardize column names based on content
                col_mapping = {}
                for std_col in standard_columns:
                    # Find the first column that contains the standard column name (case-insensitive)
                    matching_cols = [col for col in df.columns if std_col.lower() in str(col).lower()]
                    if matching_cols and matching_cols[0] not in col_mapping:
                        col_mapping[matching_cols[0]] = std_col
                    else:
                        print(f"Avertisment: Coloana '{std_col}' nu a fost găsită sau este duplicată în {file_path}. S-ar putea să existe probleme de mapare.")

                # Apply mapping and select only standard columns
                df.rename(columns=col_mapping, inplace=True)
                df = df[[col for col in standard_columns if col in df.columns]]
                
                if len(df.columns) != len(standard_columns):
                    print(f"Avertisment: După standardizare, fișierul {file_path} nu conține toate coloanele așteptate: {standard_columns}. Coloane actuale: {list(df.columns)}")

            # Ensure all standard columns are present, fill missing with NaN
            for col in standard_columns:
                if col not in df.columns:
                    df[col] = pd.NA
            df = df[standard_columns] # Reorder columns to be consistent

            # Clean up and convert data types
            df['index_id'] = pd.to_numeric(df['index_id'], errors='coerce')
            df['candidate_id'] = pd.to_numeric(df['candidate_id'], errors='coerce')
            df['candidate_id'] = df['candidate_id'].astype(int)
            df['confidence_score'] = pd.to_numeric(df['confidence_score'], errors='coerce')
            
            # Convert need_judge to boolean, handling various string representations
            df['need_judge'] = df['need_judge'].astype(str).str.lower().isin(['true', '1', 'yes'])
            
            # Drop rows where index_id is NaN (as it's a critical identifier for sorting)
            df.dropna(subset=['index_id'], inplace=True)
            
            # Convert index_id to integer type after dropping NaNs
            df['index_id'] = df['index_id'].astype(int)

            all_dataframes.append(df)
            print(f"Fișier procesat cu succes: {file_path}")

        except Exception as e:
            print(f"Eroare la procesarea fișierului {file_path}: {e}")

    if not all_dataframes:
        print("Nu există DataFrame-uri valide pentru a fi concatenate. Verificați fișierele de intrare.")
        return

    concatenated_df = pd.concat(all_dataframes, ignore_index=True)

    # Sort the final DataFrame by 'index_id'
    concatenated_df.sort_values(by='index_id', inplace=True, ascending=True)

    # Save the result
    output_path = os.path.join(folder_path, output_filename)
    concatenated_df.to_csv(output_path, index=False)
    print(f"Datele concatenate și standardizate au fost salvate în: {output_path}")

# --- Instrucțiuni de utilizare ---
# Asigurați-vă că fișierele CSV sunt plasate în același director cu acest script
# sau specificați calea absolută către directorul care conține fișierele.

# De exemplu, pentru a rula scriptul în directorul curent:
current_directory = r"C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites"
concatenate_and_standardize_csv_files(current_directory)

# Dacă fișierele se află într-un subdirector numit 'data' în cadrul directorului curent:
# data_folder_path = os.path.join(current_directory, 'data')
# concatenate_and_standardize_csv_files(data_folder_path)

# Puteți specifica și o cale absolută direct:
# specific_folder_path = "/cale/catre/directorul/cu/fisiere"
# concatenate_and_standardize_csv_files(specific_folder_path)

Avertisment: Antetul standard nu a fost găsit explicit în C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\building_places_batch_1_aligned.csv. Se încearcă inferența coloanelor.
Eroare la procesarea fișierului C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\building_places_batch_1_aligned.csv: Cannot convert non-finite values (NA or inf) to integer
Eroare la procesarea fișierului C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\creative_work_batch_1_aligned.csv: Cannot convert non-finite values (NA or inf) to integer
Fișier procesat cu succes: C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\creative_work_batch_2_aligned.csv
Avertisment: Antetul standard nu a fost găsit explicit în C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\event_batch_1_aligned.csv. Se încearcă inferența coloanelor.
Fișier procesat cu succes: C:\Users\DavidCaraman\Desktop\Licent

In [15]:
import pandas as pd
import os
import numpy as np # Import numpy for inf values

def concatenate_and_standardize_csv_files(folder_path, output_filename="entity_alignment.csv"):
    """
    Concatenates multiple CSV files from a specified folder, standardizes column names,
    sorts by 'index_id', and saves the result to a new CSV file.

    Args:
        folder_path (str): The path to the folder containing the CSV files.
        output_filename (str): The name of the output CSV file.
    """
    standard_columns = ['index_id', 'candidate_id', 'confidence_score', 'need_judge']
    all_dataframes = []

    # Get all CSV file paths in the specified folder
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.csv')]

    if not csv_files:
        print(f"Nu s-au găsit fișiere CSV în directorul: {folder_path}. Asigurați-vă că fișierele sunt prezente.")
        return

    for file_path in csv_files:
        try:
            # Read the file with no header initially to find the correct header row
            df_raw = pd.read_csv(file_path, header=None, encoding='utf-8')

            # Look for the header row containing all standard column names
            header_row_index = -1
            for i, row in df_raw.iterrows():
                row_values = row.dropna().astype(str).tolist()
                
                # Check if all standard column names (or close variations) are present
                # Use a more flexible check: any of the keywords in any of the row values
                if all(any(col.lower() in val.lower() for val in row_values) for col in standard_columns):
                    header_row_index = i
                    break
            
            if header_row_index == -1:
                print(f"Avertisment: Antetul standard nu a fost găsit explicit în {file_path}. Se încearcă inferența coloanelor.")
                # Fallback: Try to infer based on typical data start
                potential_data_start_row = -1
                for i, row in df_raw.iterrows():
                    first_val = row.dropna().iloc[0] if not row.dropna().empty else None
                    if pd.notna(first_val) and (isinstance(first_val, (int, float)) or str(first_val).replace('.', '').isdigit() or str(first_val).lower() in ['true', 'false']):
                        potential_data_start_row = i
                        break
                
                if potential_data_start_row != -1:
                    df = pd.read_csv(file_path, header=potential_data_start_row, encoding='utf-8')
                    # Assume column order based on typical CSV structure if header not found
                    if len(df.columns) >= len(standard_columns):
                        df.rename(columns={df.columns[0]: 'index_id',
                                           df.columns[1]: 'candidate_id',
                                           df.columns[2]: 'confidence_score',
                                           df.columns[3]: 'need_judge'}, inplace=True)
                        df = df[standard_columns]
                    else:
                        print(f"Se omite {file_path} din cauza numărului insuficient de coloane după inferență.")
                        continue
                else:
                    print(f"Se omite {file_path} deoarece nu a fost găsit un antet sau un început de date identificabil.")
                    continue
            else:
                # Read the file again with the identified header row
                df = pd.read_csv(file_path, header=header_row_index, encoding='utf-8')

                # Standardize column names based on content
                col_mapping = {}
                for std_col in standard_columns:
                    # Find the first column that contains the standard column name (case-insensitive)
                    matching_cols = [col for col in df.columns if std_col.lower() in str(col).lower()]
                    if matching_cols and matching_cols[0] not in col_mapping:
                        col_mapping[matching_cols[0]] = std_col
                    else:
                        print(f"Avertisment: Coloana '{std_col}' nu a fost găsită sau este duplicată în {file_path}. S-ar putea să existe probleme de mapare.")

                # Apply mapping and select only standard columns
                df.rename(columns=col_mapping, inplace=True)
                df = df[[col for col in standard_columns if col in df.columns]]
                
                if len(df.columns) != len(standard_columns):
                    print(f"Avertisment: După standardizare, fișierul {file_path} nu conține toate coloanele așteptate: {standard_columns}. Coloane actuale: {list(df.columns)}")

            # Ensure all standard columns are present, fill missing with NaN
            for col in standard_columns:
                if col not in df.columns:
                    df[col] = pd.NA
            df = df[standard_columns] # Reorder columns to be consistent

            # --- Start improved data type conversion ---
            # Convert to numeric, coercing errors and infinities to NaN
            for col in ['index_id', 'candidate_id', 'confidence_score']:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                    # Replace inf with NaN to ensure dropna catches them
                    df[col] = df[col].replace([np.inf, -np.inf], pd.NA)

            # Drop rows where index_id is NaN (as it's a critical identifier)
            df.dropna(subset=['index_id'], inplace=True)
            
            # Convert index_id to integer type (should be safe now after dropping NaNs and infs)
            df['index_id'] = df['index_id'].astype(int)

            # For candidate_id, use Int64Dtype to allow for nullable integers (if candidate_id can be missing)
            if 'candidate_id' in df.columns:
                 df['candidate_id'] = df['candidate_id'].astype(pd.Int64Dtype())
            
            # Convert need_judge to boolean
            if 'need_judge' in df.columns:
                df['need_judge'] = df['need_judge'].astype(str).str.lower().isin(['true', '1', 'yes'])
            # --- End improved data type conversion ---

            all_dataframes.append(df)
            print(f"Fișier procesat cu succes: {file_path}")

        except Exception as e:
            print(f"Eroare la procesarea fișierului {file_path}: {e}")

    if not all_dataframes:
        print("Nu există DataFrame-uri valide pentru a fi concatenate. Verificați fișierele de intrare.")
        return

    concatenated_df = pd.concat(all_dataframes, ignore_index=True)

    # Sort the final DataFrame by 'index_id'
    concatenated_df.sort_values(by='index_id', inplace=True, ascending=True)

    # Save the result
    output_path = os.path.join(folder_path, output_filename)
    concatenated_df.to_csv(output_path, index=False)
    print(f"Datele concatenate și standardizate au fost salvate în: {output_path}")

# --- Instrucțiuni de utilizare ---
# Asigurați-vă că fișierele CSV sunt plasate în același director cu acest script
# sau specificați calea absolută către directorul care conține fișierele.

# De exemplu, pentru a rula scriptul în directorul curent:
current_directory = r"C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites"
concatenate_and_standardize_csv_files(current_directory)

# Dacă fișierele se află într-un subdirector numit 'data' în cadrul directorului curent:
# data_folder_path = os.path.join(current_directory, 'data')
# concatenate_and_standardize_csv_files(data_folder_path)

# Puteți specifica și o cale absolută direct:
# specific_folder_path = "/cale/catre/directorul/cu/fisiere"
# concatenate_and_standardize_csv_files(specific_folder_path)

Avertisment: Antetul standard nu a fost găsit explicit în C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\building_places_batch_1_aligned.csv. Se încearcă inferența coloanelor.
Fișier procesat cu succes: C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\building_places_batch_1_aligned.csv
Fișier procesat cu succes: C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\creative_work_batch_1_aligned.csv
Fișier procesat cu succes: C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\creative_work_batch_2_aligned.csv
Avertisment: Antetul standard nu a fost găsit explicit în C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\event_batch_1_aligned.csv. Se încearcă inferența coloanelor.
Fișier procesat cu succes: C:\Users\DavidCaraman\Desktop\Licenta 1.0\mvp-kg-alignment\aligned_enitites\event_batch_1_aligned.csv
Fișier procesat cu succes: C:\Users\DavidCaraman\Desktop\L